In [10]:
#!/usr/bin/env python -W ignore::DeprecationWarning
import pickle
import _pickle as cPickle
import codecs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pylab as pl
import seaborn as sns
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import os

import gensim
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.similarities import WmdSimilarity
from gensim.models import KeyedVectors
if not os.path.exists('data/GoogleNews-vectors-negative300.bin.gz'):
    raise ValueError("SKIP: You need to download the google news model")
from gensim.models import Doc2Vec
from random import shuffle
from sklearn.linear_model import LogisticRegression

from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis

from sner import Ner
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag.stanford import CoreNLPPOSTagger 

# Change the path according to your system
stanford_classifier = 'http://localhost:9199'
stanford_pos_tagger = 'http://localhost:9000'
NERst = Ner(host='localhost',port=9199)

from pyemd import emd 

import re

from nltk.stem import SnowballStemmer
stop_words = stopwords.words('english')
_stemmer = SnowballStemmer('english')

In [11]:
import random
import itertools
import matplotlib.mlab as mlab

from collections import Counter
from sklearn.model_selection import train_test_split

from keras.models import Model
from keras.layers import Embedding
from keras.layers import Lambda
from keras.layers.merge import concatenate
from keras.layers import LSTM, Bidirectional
from keras.layers import Input, Dense, Dropout
from keras.layers import Convolution1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint

import spacy
nlp = spacy.load('en_core_web_lg');


In [12]:
from nltk.corpus import wordnet as wn
from nltk import pos_tag

In [13]:
import time

In [14]:
def preprocess(df):
    df = clean_text_df(df)
    # df = stem(df)
    return df

In [15]:
def stem(df):
    """
    Process the text data with SnowballStemmer
    :param df: dataframe of original data
    :return: dataframe after stemming
    """
    df['question1'] = df.question1.map(lambda x: ' '.join(
        [_stemmer.stem(word) for word in
         nltk.word_tokenize(clean_text(str(x).lower()))]))
    df['question2'] = df.question2.map(lambda x: ' '.join(
        [_stemmer.stem(word) for word in
         nltk.word_tokenize(clean_text(str(x).lower()))]))
    return df

In [16]:
def clean_text_df(df):
    """
    Process the text data with SnowballStemmer
    :param df: dataframe of original data
    :return: dataframe after stemming
    """
    df['question1'] = df.question1.map(lambda x: clean_text(str(x).lower()))
    df['question2'] = df.question2.map(lambda x: clean_text(str(x).lower()))
    return df

In [19]:
def clean_text(x):
    """
    Clean text
    :param text: the string of text
    :return: text string after cleaning
    """
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'").replace("\“", "\"").replace("\”", "\"")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will").replace(" fb ", " facebook ")\
                           .replace("facebooks", " facebook ").replace("insidefacebook", "inside facebook")\
                           .replace("=", " equal ").replace("pokemons", "pokemon").replace("pokémon", "pokemon")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    # remove extra space
    x = ' '.join(x.split())

    return x

In [20]:
path = "data/"
train_file_name = "train";
test_file_name = "test";
train_data = pd.read_csv(path+train_file_name+'.csv')
test_data = pd.read_csv(path+test_file_name+'.csv')

# train_data = pd.read_csv(path+'quora_train_features.csv',encoding = "utf-8")
# test_data = pd.read_csv(path+'quora_test_features.csv',encoding = "utf-8")
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

In [21]:
train_data = preprocess(train_data)
test_data = preprocess(test_data)

In [22]:
def wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    s1 = [w for w in s1 if w not in stop_words]
    s1 = [w for w in s1 if w.isalpha()]
    s2 = [w for w in s2 if w not in stop_words]
    s2 = [w for w in s2 if w.isalpha()]
    return model.wmdistance(s1, s2)

In [23]:
def norm_wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    s1 = [w for w in s1 if w not in stop_words]
    s1 = [w for w in s1 if w.isalpha()]
    s2 = [w for w in s2 if w not in stop_words]
    s2 = [w for w in s2 if w.isalpha()]
    return norm_model.wmdistance(s1, s2)

In [24]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

In [25]:
def w2v(data, norm):
    if (norm):
        norm_model.init_sims(replace=True)
        data['norm_wmd'] = data.apply(lambda x: norm_wmd(x['question1'], x['question2']), axis=1)
    else:
        data['w2v'] = data.apply(lambda x: wmd(str(x['question1']), str(x['question2'])), axis=1)
    return data

In [26]:
# https://www.codeproject.com/Articles/11835/WordNet-based-semantic-similarity-measurement
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
    return None

def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0
 
    # For each word in the first sentence
    for syn1 in synsets1:
        arr_simi_score = []
        for syn2 in synsets2:
            simi_score = syn1.path_similarity(syn2)
            if simi_score is not None:
                arr_simi_score.append(simi_score)
            else:
                arr_simi_score.append(0)
        if(len(arr_simi_score) > 0):
            best = max(arr_simi_score)
            score += best
            count += 1
    if count != 0:
        score /= count
    return score

def symmetric_sentence_similarity(index,sentence1, sentence2):
    if( len(sentence1)==0 or len(sentence2) == 0):
            return 0
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_similarity(sentence1, sentence2) + sentence_similarity(sentence2, sentence1)) / 2


In [27]:
def semantic_sentence_similarity(index, sentence1, sentence2):
    if( int(index)%100000 == 0):
        print(time.strftime("%c"))
        print(index)
    return nlp(sentence1).similarity(nlp(sentence2))

In [28]:
def feature(data):
    data['len_q1'] = data.question1.apply(lambda x: len(str(x)))
    data['len_q2'] = data.question2.apply(lambda x: len(str(x)))
    data['diff_len'] = data.len_q1 - data.len_q2
    data['len_unique_char_q1'] = data.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
    data['len_unique_char_q2'] = data.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
    data['len_word_q1'] = data.question1.apply(lambda x: len(str(x).split()))
    data['len_word_q2'] = data.question2.apply(lambda x: len(str(x).split()))
    data['common_words'] = data.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
    data['fuzz_qratio'] = data.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
    data['fuzz_WRatio'] = data.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
    data['fuzz_partial_ratio'] = data.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
#    data['fuzz_partial_token_set_ratio'] = data.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
    data['fuzz_partial_token_sort_ratio'] = data.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
    data['fuzz_token_set_ratio'] = data.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
    data['fuzz_token_sort_ratio'] = data.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
    
    return data

In [29]:
def feature_2(data):
    data['semantic_similarity'] = data.apply(lambda x: semantic_sentence_similarity( x.name,str(x['question1']), str(x['question2'])) , axis=1)
    return data

In [30]:
def feature_3(data):
    question1_vectors = np.zeros((data.shape[0], 300))
    error_count = 0
    for i, q in tqdm(enumerate(data.question1.values)):
        question1_vectors[i, :] = sent2vec(q)
    question2_vectors  = np.zeros((data.shape[0], 300))
    for i, q in tqdm(enumerate(data.question2.values)):
        question2_vectors[i, :] = sent2vec(q)
    data['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                              np.nan_to_num(question2_vectors))]
    data['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                              np.nan_to_num(question2_vectors))]
    data['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                              np.nan_to_num(question2_vectors))]
    data['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                              np.nan_to_num(question2_vectors))]
    data['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                              np.nan_to_num(question2_vectors))]
    data['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                              np.nan_to_num(question2_vectors))]
    data['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                              np.nan_to_num(question2_vectors))]
    data['skew_q1vec'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
    data['skew_q2vec'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
    data['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
    data['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]
    cPickle.dump(question1_vectors, open('data/q1_w2v.pkl', 'wb'), -1)
    cPickle.dump(question2_vectors, open('data/q2_w2v.pkl', 'wb'), -1)
    return data

In [31]:
def feature_4(data):
    data['sentence_similarity'] = data.apply(lambda x: symmetric_sentence_similarity(x.name, str(x['question1']), str(x['question2'])), axis=1)
    return data

In [55]:
from collections import defaultdict
ques = pd.concat([train_data[['question1', 'question2']],
                  test_data[['question1', 'question2']]], axis=0).reset_index(drop='index')
ques.shape

def q1_freq(row):
    return (len(q_dict[row['question1']]))

def q2_freq(row):
    return (len(q_dict[row['question2']]))

def q1_q2_intersect(row):
    return(len(set(q_dict[row['question1']]).intersection(set(q_dict[row['question2']]))))

def feature_5(df):
    for i in range(ques.shape[0]):
        q_dict[ques.question1[i]].add(ques.question2[i])
        q_dict[ques.question2[i]].add(ques.question1[i])
    df['q1_q2_intersect'] = df.apply(q1_q2_intersect, axis=1, raw=True)
    df['q1_freq'] = df.apply(q1_freq, axis=1, raw=True)
    df['q2_freq'] = df.apply(q2_freq, axis=1, raw=True)
    return df

In [33]:

def word_match_share(q1, q2, stop_words=None):
    q1 = str(q1).lower().split()
    q2 = str(q2).lower().split()
    q1words = {}
    q2words = {}
    for word in q1:
        if word not in stop_words:
            q1words[word] = 1
    for word in q2:
        if word not in stop_words:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0.
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

In [34]:
def feature_6(row):
    for i in range(ques.shape[0]):
        wm = word_match_share(ques.question1[i], ques.question2[i], stop_words=stop_words)
        q_dict[ques.question1[i]][ques.question2[i]] = wm
        q_dict[ques.question2[i]][ques.question1[i]] = wm
        q1 = q_dict[row['question1']]
        q2 = q_dict[row['question2']]
    inter_keys = set(q1.keys()).intersection(set(q2.keys()))
    if(len(inter_keys) == 0): return 0.
    inter_wm = 0.
    total_wm = 0.
    for q,wm in q1.items():
        if q in inter_keys:
            inter_wm += wm
        total_wm += wm
    for q,wm in q2.items():
        if q in inter_keys:
            inter_wm += wm
        total_wm += wm
    if(total_wm == 0.): return 0.
    return inter_wm/total_wm

In [35]:
import distance
SAFE_DIV = 0.0001
# also pre-processing: tokennizing questions, removing empty q's, seperating stop words， then find all features
def get_token_features(q1, q2):
    token_features = [0.0]*10

    q1_tokens = q1.split()
    q2_tokens = q2.split()

    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    q1_words = set([word for word in q1_tokens if word not in stop_words])
    q2_words = set([word for word in q2_tokens if word not in stop_words])

    q1_stops = set([word for word in q1_tokens if word in stop_words])
    q2_stops = set([word for word in q2_tokens if word in stop_words])

    common_word_count = len(q1_words.intersection(q2_words))
    common_stop_count = len(q1_stops.intersection(q2_stops))
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))

# feature extraction
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)  # check ratio between common and smaller(processed)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)  # check ratio between common and larger(processed)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)  # check ratio between common and smaller(stop words)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)  # check ratio between common and larger(stop words)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV) # check ratio between common and smaller(total)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV) # check ratio between common and larger(total)
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1]) # check if endings are the same
    token_features[7] = int(q1_tokens[0] == q2_tokens[0]) # check if beginnings are the same
    token_features[8] = abs(len(q1_tokens) - len(q2_tokens)) # check diff between line
    token_features[9] = (len(q1_tokens) + len(q2_tokens))/2 # check average length
    return token_features


def get_longest_substr_ratio(a, b):
    strs = list(distance.lcsubstrings(a, b))
    if len(strs) == 0:
        return 0
    else:
        return len(strs[0]) / (min(len(a), len(b)) + 1)
    
def feature_7(data):
    token_features = data.apply(lambda x: get_token_features(str(x["question1"]), str(x["question2"])), axis=1)
    data["cwc_min"]       = list(map(lambda x: x[0], token_features))
    data["cwc_max"]       = list(map(lambda x: x[1], token_features))
    data["csc_min"]       = list(map(lambda x: x[2], token_features))
    data["csc_max"]       = list(map(lambda x: x[3], token_features))
    data["ctc_min"]       = list(map(lambda x: x[4], token_features))
    data["ctc_max"]       = list(map(lambda x: x[5], token_features))
    data["last_word_eq"]  = list(map(lambda x: x[6], token_features))
    data["first_word_eq"] = list(map(lambda x: x[7], token_features))
    data["abs_len_diff"]  = list(map(lambda x: x[8], token_features))
    data["mean_len"]      = list(map(lambda x: x[9], token_features))
    data["longest_substr_ratio"]  = data.apply(lambda x: get_longest_substr_ratio(str(x["question1"]), str(x["question2"])), axis=1)
    return data

In [36]:
import networkx as nx

NB_CORES = 10
FREQ_UPPER_BOUND = 100
NEIGHBOR_UPPER_BOUND = 5


def create_question_hash(train_df, test_df):
    train_qs = np.dstack([train_df["question1"], train_df["question2"]]).flatten() # adds all training questions into one list
    test_qs = np.dstack([test_df["question1"], test_df["question2"]]).flatten() # adds all testing questions into one list
    all_qs = np.append(train_qs, test_qs) # all questions together
    all_qs = pd.DataFrame(all_qs)[0].drop_duplicates() # all unique questions
    all_qs.reset_index(inplace=True, drop=True) # create a new table with all questions
    question_dict = pd.Series(all_qs.index.values, index=all_qs.values).to_dict()
    return question_dict


def get_hash(df, hash_dict):
    df["qid1"] = df["question1"].map(hash_dict)
    df["qid2"] = df["question2"].map(hash_dict)
    return df


def get_kcore_dict(df):
    g = nx.Graph()
    g.add_nodes_from(df.qid1)
    edges = list(df[["qid1", "qid2"]].to_records(index=False))
    g.add_edges_from(edges)
    g.remove_edges_from(g.selfloop_edges())

    df_output = pd.DataFrame(data=list(g.nodes()), columns=["qid"])
    df_output["kcore"] = 0
    for k in range(2, NB_CORES + 1):
        ck = nx.k_core(g, k=k).nodes()
        print("kcore", k)
        df_output.ix[df_output.qid.isin(ck), "kcore"] = k

    return df_output.to_dict()["kcore"]


def get_kcore_features(df, kcore_dict):
    df["kcore1"] = df["qid1"].apply(lambda x: kcore_dict[x])
    df["kcore2"] = df["qid2"].apply(lambda x: kcore_dict[x])
    return df


def convert_to_minmax(df, col):
    sorted_features = np.sort(np.vstack([df[col + "1"], df[col + "2"]]).T)
    df["min_" + col] = sorted_features[:, 0]
    df["max_" + col] = sorted_features[:, 1]
    return df.drop([col + "1", col + "2"], axis=1)


def get_neighbors(train_df, test_df):
    neighbors = defaultdict(set)
    for df in [train_df, test_df]:
        for q1, q2 in zip(df["qid1"], df["qid2"]):
            neighbors[q1].add(q2)
            neighbors[q2].add(q1)
    return neighbors


def get_neighbor_features(df, neighbors):
    common_nc = df.apply(lambda x: len(neighbors[x.qid1].intersection(neighbors[x.qid2])), axis=1)
    min_nc = df.apply(lambda x: min(len(neighbors[x.qid1]), len(neighbors[x.qid2])), axis=1)
    df["common_neighbor_ratio"] = common_nc / min_nc
    df["common_neighbor_count"] = common_nc.apply(lambda x: min(x, NEIGHBOR_UPPER_BOUND))
    return df


def get_freq_features(df, frequency_map):
    df["freq1"] = df["qid1"].map(lambda x: min(frequency_map[x], FREQ_UPPER_BOUND))
    df["freq2"] = df["qid2"].map(lambda x: min(frequency_map[x], FREQ_UPPER_BOUND))
    return df


In [37]:
train_data = feature(train_data)

In [38]:
test_data = feature(test_data)

In [39]:
train_data = feature_2(train_data)

Sun Dec 10 00:21:08 2017
0
Sun Dec 10 00:51:50 2017
100000
Sun Dec 10 01:22:15 2017
200000
Sun Dec 10 01:52:29 2017
300000
Sun Dec 10 02:22:42 2017
400000


In [40]:
test_data = feature_2(test_data)

Sun Dec 10 02:24:01 2017
0
Sun Dec 10 02:54:16 2017
100000
Sun Dec 10 03:24:33 2017
200000
Sun Dec 10 03:54:36 2017
300000
Sun Dec 10 04:24:48 2017
400000
Sun Dec 10 04:55:02 2017
500000
Sun Dec 10 05:25:17 2017
600000
Sun Dec 10 05:55:34 2017
700000
Sun Dec 10 06:25:51 2017
800000
Sun Dec 10 06:56:10 2017
900000
Sun Dec 10 07:26:27 2017
1000000
Sun Dec 10 07:56:46 2017
1100000
Sun Dec 10 08:27:04 2017
1200000
Sun Dec 10 08:57:23 2017
1300000
Sun Dec 10 09:27:43 2017
1400000
Sun Dec 10 09:58:03 2017
1500000
Sun Dec 10 10:28:22 2017
1600000
Sun Dec 10 10:58:42 2017
1700000
Sun Dec 10 11:29:01 2017
1800000
Sun Dec 10 11:59:20 2017
1900000
Sun Dec 10 12:29:46 2017
2000000
Sun Dec 10 13:00:44 2017
2100000
Sun Dec 10 13:31:42 2017
2200000
Sun Dec 10 14:02:43 2017
2300000


In [42]:
model = KeyedVectors.load_word2vec_format('data/glove.840B.300d.txt.word2vec', binary=False)

In [43]:
norm_model = gensim.models.KeyedVectors.load_word2vec_format('data/glove.840B.300d.txt.word2vec', binary=False)
norm_model.init_sims(replace=True)

In [44]:
train_data = w2v(train_data, True)

In [45]:
train_data = w2v(train_data, False)

In [46]:
test_data = w2v(test_data, True)

In [47]:
test_data = w2v(test_data, False) 

In [48]:
train_data = feature_3(train_data)

0it [00:00, ?it/s]c:\program files\python36\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
404290it [01:01, 6607.25it/s]
404290it [01:02, 6492.79it/s]
c:\program files\python36\lib\site-packages\scipy\spatial\distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
c:\program files\python36\lib\site-packages\scipy\spatial\distance.py:763: RuntimeWarning: invalid value encountered in double_scalars
  dist = np.double(unequal_nonzero.sum()) / np.double(nonzero.sum())
c:\program files\python36\lib\site-packages\scipy\spatial\distance.py:980: RuntimeWarning: invalid value encountered in double_scalars
  return l1_diff.sum() / l1_sum.sum()


In [49]:
test_data = feature_3(test_data)

0it [00:00, ?it/s]c:\program files\python36\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
2345796it [06:03, 6455.75it/s]
2345796it [06:12, 6292.99it/s]
c:\program files\python36\lib\site-packages\scipy\spatial\distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
c:\program files\python36\lib\site-packages\scipy\spatial\distance.py:763: RuntimeWarning: invalid value encountered in double_scalars
  dist = np.double(unequal_nonzero.sum()) / np.double(nonzero.sum())
c:\program files\python36\lib\site-packages\scipy\spatial\distance.py:980: RuntimeWarning: invalid value encountered in double_scalars
  return l1_diff.sum() / l1_sum.sum()


In [50]:
train_data = feature_4(train_data)

In [51]:
test_data = feature_4(test_data)

In [ ]:
q_dict = defaultdict(set)

train_data= feature_5(train_data)
test_data= feature_5(test_data)

In [ ]:
# q_dict = defaultdict(dict)
# train_data['q1_q2_wm_ratio'] = train_data.apply(feature_6, axis=1, raw=True)
# test_data['q1_q2_wm_ratio'] = test_data.apply(feature_6, axis=1, raw=True)

In [ ]:
print("Hashing the questions...")
question_dict = create_question_hash(train_data, test_data)
train_data = get_hash(train_data, question_dict)
test_data = get_hash(test_data, question_dict)

In [ ]:
print("Calculating kcore features...")
all_df = pd.concat([train_data, test_data])
kcore_dict = get_kcore_dict(all_df)
train_data = get_kcore_features(train_data, kcore_dict)
test_data = get_kcore_features(test_data, kcore_dict)
train_data = convert_to_minmax(train_data, "kcore")
test_data = convert_to_minmax(test_data, "kcore")

In [ ]:
print("Calculating common neighbor features...")
neighbors = get_neighbors(train_data, test_data)
train_data = get_neighbor_features(train_data, neighbors)
test_data = get_neighbor_features(test_data, neighbors)

In [ ]:
print("Calculating frequency features...")
frequency_map = dict(zip(*np.unique(np.vstack((all_df["qid1"], all_df["qid2"])), return_counts=True)))
train_data = get_freq_features(train_data, frequency_map)
test_data = get_freq_features(test_data, frequency_map)
train_data = convert_to_minmax(train_data, "freq")
test_data = convert_to_minmax(test_data, "freq")

In [ ]:
train_data = feature_7(train_data)

In [ ]:
test_data = feature_7(test_data)

In [ ]:
df_train = pd.read_csv('data/train.csv', 
                       dtype={
                           'question1': np.str,
                           'question2': np.str
                       })
df_test = pd.read_csv('data/test.csv', 
                      dtype={
                          'question1': np.str,
                          'question2': np.str
                      })
df_train = df_train.fillna(' ')
df_test = df_test.fillna(' ')

In [ ]:
qs = set(df_train['question1']) | set(df_train['question2']) | set(df_test['question1']) | set(df_test['question2'])
qs = list(qs)
q2id = dict(zip(qs, range(len(qs))))

In [ ]:
df_train['qid1'] = df_train['question1'].apply(q2id.get)
df_train['qid2'] = df_train['question2'].apply(q2id.get)
df_test['qid1'] = df_test['question1'].apply(q2id.get)
df_test['qid2'] = df_test['question2'].apply(q2id.get)

In [ ]:
from scipy.sparse import coo_matrix, csr_matrix

m = coo_matrix((np.ones(2 * (df_train.shape[0] + df_test.shape[0])),
                (pd.concat((df_train['qid1'], df_test['qid1'], df_train['qid2'], df_test['qid2']), axis=0).values,
                 pd.concat((df_train['qid2'], df_test['qid2'], df_train['qid1'], df_test['qid1']), axis=0).values)),
               shape=(len(qs), len(qs)))
m = csr_matrix(m)

In [ ]:
from sklearn.preprocessing import normalize

def cosine_similarities(mat):
    col_normed_mat = normalize(mat.tocsc(), axis=0)
    return col_normed_mat.T * col_normed_mat

def jaccard_similarities(mat):
    cols_sum = mat.getnnz(axis=0)
    ab = mat.T * mat

    # for rows
    aa = np.repeat(cols_sum, ab.getnnz(axis=0))
    # for columns
    bb = cols_sum[ab.indices]

    similarities = ab.copy()
    similarities.data /= (aa + bb - ab.data)

    return similarities

In [ ]:
%%time
m_cos = cosine_similarities(m)
m_jac = jaccard_similarities(m)

In [ ]:
m_cos = m_cos.todok()
m_jac = m_jac.todok()

In [ ]:
%%time
def devil_cos(row):
    return m_cos[row['qid1'], row['qid2']]

def devil_jaccard(row):
    return m_jac[row['qid1'], row['qid2']]

df_train['devil_cos'] = df_train.apply(devil_cos, axis=1, raw=True)
df_train['devil_jaccard'] = df_train.apply(devil_jaccard, axis=1, raw=True)
df_test['devil_cos'] = df_test.apply(devil_cos, axis=1, raw=True)
df_test['devil_jaccard'] = df_test.apply(devil_jaccard, axis=1, raw=True)

In [ ]:
df_train = df_train.drop(['id','qid1','qid2','question1','question2','is_duplicate'],axis = 1)

In [ ]:
df_test = df_test.drop(['test_id','qid1','qid2','question1','question2'],axis = 1)

In [ ]:
train_data = pd.concat([train_data,df_train], axis=1)

In [ ]:
test_data = pd.concat([test_data,df_test], axis=1)

In [ ]:
train_data.to_csv(path+'quora_train_features.csv', index=False,encoding='utf-8')

In [ ]:
test_data.to_csv(path+'quora_test_features.csv', index=False,encoding='utf-8')

In [ ]:
# cleared_train = pd.read_csv(path+'quora_train_features.csv')
# cleared_test = pd.read_csv(path+'quora_train_features.csv')
# cleared_train = pd.read_csv(path+'quora_train_features_cleared.csv')
# cleared_test = pd.read_csv(path+'quora_test_features_cleared.csv')

In [ ]:
cleared_train = train_data
cleared_test = test_data

In [ ]:
cleared_train = cleared_train.drop(['id','qid1','qid2','question1','question2','is_duplicate'],axis = 1)

In [ ]:
cleared_test = cleared_test.drop(['test_id','qid1','qid2','question1','question2'],axis = 1)

In [ ]:
# cleared_test.columns.values.tolist()

In [ ]:
cleared_train.to_csv('data/quora_train_features_cleared.csv', index=False,encoding='utf-8')

In [ ]:
cleared_test.to_csv('data/quora_test_features_cleared.csv', index=False,encoding='utf-8')

In [ ]:
# cleared_train = pd.read_csv(path+'quora_train_features_cleared_best.csv')
# cleared_test = pd.read_csv(path+'quora_test_features_cleared_best.csv')

In [ ]:
# cleared_train = cleared_train.drop(['q1_q2_wm_ratio','common_neighbor_ratio'],axis=1)

In [ ]:
# cleared_test = cleared_test.drop(['q1_q2_wm_ratio','common_neighbor_ratio'],axis=1)

In [ ]:
# cleared_train = cleared_train.drop(['fuzz_partial_token_set_ratio'],axis=1)

In [ ]:
# cleared_test = cleared_test.drop(['fuzz_partial_token_set_ratio'],axis=1)

In [ ]:
ground_truth = train_data['is_duplicate'].values

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [ ]:
cleared_train = cleared_train.replace(np.nan, 0.)
cleared_train = cleared_train.replace(np.inf, 99.)
cleared_test = cleared_test.replace(np.nan, 0.)
cleared_test = cleared_test.replace(np.inf, 99.)

In [ ]:
cleared_train.isnull().values.any()

In [ ]:
import xgboost as xgb
# read in data

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(cleared_train, ground_truth, test_size=0.1, random_state=17)


In [ ]:
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 7
params['subsample'] = 0.75
params['base_score'] = 0.2

params['colsample_bytree'] = 1
params['colsample_bylevel'] = 1
params['n_jobs'] = -1

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid)

In [ ]:
d_test = xgb.DMatrix(cleared_test)

In [ ]:
watchlist = [(d_train, 'train'), (d_valid, 'valid')]


In [ ]:
bst = xgb.train(params, d_train, 5000, watchlist, early_stopping_rounds=100, verbose_eval=100)

In [ ]:
p_test = bst.predict(d_test)
p_test

In [ ]:
a = 0.165 / 0.37
b = (1 - 0.165) / (1 - 0.37)

def fix_prob(x):
    return a * x / (a * x + b * (1 - x))

In [ ]:
sub = pd.DataFrame()
sub['test_id'] = test_data.index.values
sub['is_duplicate'] = fix_prob(p_test)

In [ ]:
sub.to_csv('data/simple_xgb2.csv', index=False)

In [ ]:

import numpy as np
import pandas as pd

df_train =  pd.read_csv('data/train.csv')
df_test =  pd.read_csv('data/test.csv')

In [ ]:
#REPLACE it with:
test_label = np.array(pd.read_csv('data/simple_xgb2_test.csv')["is_duplicate"])

In [ ]:
from collections import defaultdict

REPEAT = 2 #a reasonable number which can consider your updates iteratively but not ruin the predictions

DUP_THRESHOLD = 0.5 #classification threshold for duplicates
NOT_DUP_THRESHOLD = 0.1 #classification threshold for non-duplicates
#Since the data is unbalanced, our mean prediction is around 0.16. So this is the reason of unbalanced thresholds

MAX_UPDATE = 0.2 # maximum update on the dup probability (a high choice may ruin the predictions)
DUP_UPPER_BOUND = 0.98 # do not update dup probabilities above this threshold
NOT_DUP_LOWER_BOUND = 0.01 # do not update dup probabilities below this threshold
# There is no significant gain between 0.98 and 1.00 for a dup 
# but there is significant loss if it is not really a dup

In [ ]:
for i in range(REPEAT):
    dup_neighbors = defaultdict(set)

    for dup, q1, q2 in zip(df_train["is_duplicate"], df_train["question1"], df_train["question2"]): 
        if dup:
            dup_neighbors[q1].add(q2)
            dup_neighbors[q2].add(q1)
    
    for dup, q1, q2 in zip(test_label, df_test["question1"], df_test["question2"]): 
        if dup > DUP_THRESHOLD:
            dup_neighbors[q1].add(q2)
            dup_neighbors[q2].add(q1)

    count = 0
    for index, (q1, q2) in enumerate(zip(df_test["question1"], df_test["question2"])): 
        dup_neighbor_count = len(dup_neighbors[q1].intersection(dup_neighbors[q2]))
        if dup_neighbor_count > 0 and test_label[index] < DUP_UPPER_BOUND:
            update = min(MAX_UPDATE, (DUP_UPPER_BOUND - test_label[index])/2)
            test_label[index] += update
            count += 1

    print("Edited:", count)

In [ ]:
for i in range(REPEAT):
    not_dup_neighbors = defaultdict(set)

    for dup, q1, q2 in zip(df_train["is_duplicate"], df_train["question1"], df_train["question2"]): 
        if not dup:
            not_dup_neighbors[q1].add(q2)
            not_dup_neighbors[q2].add(q1)
    
    for dup, q1, q2 in zip(test_label, df_test["question1"], df_test["question2"]): 
        if dup < NOT_DUP_THRESHOLD:
            not_dup_neighbors[q1].add(q2)
            not_dup_neighbors[q2].add(q1)

    count = 0
    for index, (q1, q2) in enumerate(zip(df_test["question1"], df_test["question2"])): 
        dup_neighbor_count = len(not_dup_neighbors[q1].intersection(not_dup_neighbors[q2]))
        if dup_neighbor_count > 0 and test_label[index] > NOT_DUP_LOWER_BOUND:
            update = min(MAX_UPDATE, (test_label[index] - NOT_DUP_LOWER_BOUND)/2)
            test_label[index] -= update
            count += 1

    print("Edited:", count)

In [ ]:
submission = pd.DataFrame({'test_id':df_test["test_id"], 'is_duplicate':test_label})
submission.to_csv('data/submission.csv', index=False)

In [ ]:
sub.to_csv('data/simple_xgb2.csv', index=False)

In [ ]:
sub

In [ ]:
import eli5
eli5.explain_weights_xgboost(bst, top=50)